# Multi-Objective Knapsack Problem

Extends the classic Knapsack Problem by introducing multiple conflicting objectives to optimize simultaneously. This problem is of interest when you have several criteria to consider when selecting items for the knapsack, and these criteria may conflict with each other.

In the Multi-Objective Knapsack Problem:

- You are given a set of items, each with multiple attributes (e.g., weight, value, and one or more additional attributes).
- You have a knapsack with limited capacity.
- There are multiple objectives to optimize, such as maximizing the total value, minimizing the total weight, and optimizing other attributes simultaneously.

The challenge in MOKP is to find a set of items to include in the knapsack that provides a trade-off between the conflicting objectives. It's common to use evolutionary algorithms, such as multi-objective genetic algorithms, to solve this problem.

## MOKP - Robot Fleet

You are in charge of selecting a team of robots for a space mission. You have a limited energy reserve on your spacecraft, and you need to decide which robots to take with you. Each robot has three attributes: energy consumption, repairability, and task completion time.

- Energy Consumption (EC): The amount of energy a robot consumes during the mission. Lower values are preferred as they reduce the need for recharging or refueling.

- Repairability (R): A measure of how easily a robot can be repaired if it malfunctions during the mission. Higher values indicate greater repairability.

- Task Completion Time (TCT): The time it takes for a robot to complete its assigned tasks. Shorter task completion times are preferred for efficiency.

You have a fixed energy reserve (C) for your spacecraft, and you want to maximize the following objectives:

1. Maximize the total repairability (sum of repairability values) of the selected robots.
2. Minimize the total energy consumption (sum of energy consumption values) of the selected robots.
3. Minimize the total task completion time (sum of task completion time values) of the selected robots.

Formally, the Multi-Objective Knapsack Problem can be described as follows:

- Given a set of n robots, each with attributes (EC_i, R_i, TCT_i), where i = 1 to n.
- Given a spacecraft with a energy reserve (C).
- Find a subset of robots to include in the mission that maximizes repairability, minimizes energy consumption, and minimizes task completion time, while ensuring that the total energy consumption (sum of EC_i) of the selected robots does not exceed the energy reserve (C).


In [176]:
import random
from collections import namedtuple
import numpy as np

In [177]:
# evolution parameters
POPULATION_SIZE = 50
MUTATION_RATE = 0.05
NUMBER_GENERATIONS = 200

# problem parameters
NUMBER_ROBOT = 20
MAX_EC = 100  # energy req
MAX_R = 100  # reliability
MAX_TCT = 100  # time to task completion
ENERGY_RESERVE = 1000  # max energy reserve of the ship

In [178]:
# define named tuple for the robot
Robot = namedtuple("Robot", ["ec", "r", "tct"])

# build robots inventory
robots = [
    Robot(
        random.randint(1, MAX_EC),
        random.randint(1, MAX_R),
        random.randint(1, MAX_TCT),
    )
    for _ in range(NUMBER_ROBOT)
]

# initialize population -> genome is a list of 0 and 1 (genes) (0: robot not selected, 1: robot selected)
population = [
    [random.randint(0, 1) for _ in range(NUMBER_ROBOT)] for _ in range(POPULATION_SIZE)
]

In [179]:
# min(sum(ec)), max(sum(r)), min(sum(tct)) while sum(ec) <= ENERGY_RESERVE
# maximize reliability and minimize time to task completion and total energy -> max(r/tct) while sum(ec) <= ENERGY_RESERVE
def fitness1(genome):
    count = genome.count(1)
    ec = 0
    r = 0
    tct = 0
    for i, gene in enumerate(genome):
        if gene == 1:
            ec += robots[i].ec
            r += robots[i].r
            tct += robots[i].tct
    # penalize if energy reserve is exceeded or genome is all 0
    if ec > ENERGY_RESERVE or count == 0:
        return 0
    return r / tct

In [180]:
W_EC = -0.5
W_R = 0.3
W_TCT = -0.2

def fitness2(genome):
    # linear combo of weights and vars
    count = genome.count(1)
    ec = 0
    r = 0
    tct = 0
    for i, gene in enumerate(genome):
        if gene == 1:
            ec += robots[i].ec
            r += robots[i].r
            tct += robots[i].tct

    fitness = W_EC * ec + W_R * r + W_TCT * tct
    if ec > ENERGY_RESERVE or fitness <= 0:
        return 0
    return fitness

In [181]:
def run(fitness_function, population):
    for generation in range(NUMBER_GENERATIONS):
        # fitness eval
        fitness_scores = [fitness_function(genome) + 0.01 for genome in population]

        # parents selection ! with fitness2 fitness can be negative
        selected_parents = random.choices(
            population, weights=fitness_scores, k=POPULATION_SIZE
        )

        new_population = []
        for _ in range(POPULATION_SIZE // 2):
            parent1, parent2 = random.choices(selected_parents, k=2)

            # recombination
            child1 = parent1[: NUMBER_ROBOT // 2] + parent2[NUMBER_ROBOT // 2 :]
            child2 = parent2[: NUMBER_ROBOT // 2] + parent1[NUMBER_ROBOT // 2 :]

            # mutation
            child1 = [
                1 - gene if random.random() < MUTATION_RATE else gene for gene in child1
            ]
            child2 = [
                1 - gene if random.random() < MUTATION_RATE else gene for gene in child2
            ]  # chance of mutation ie invert gene

            new_population.append(child1)
            new_population.append(child2)

        population = new_population

    # best solution
    best_chromosome = max(population, key=fitness1)
    print()
    print("Fitness function:", fitness_function.__name__)
    print("Best solution:", best_chromosome)
    print("Fitness score:", fitness1(best_chromosome))
    print(
        "Total energy:",
        sum([robots[i].ec for i, gene in enumerate(best_chromosome) if gene == 1]),
    )
    print(
        "Mean Reliability /100:",
        np.mean([robots[i].r for i, gene in enumerate(best_chromosome) if gene == 1]),
    )
    print(
        "Mean Time to task completion /100:",
        np.mean([robots[i].tct for i, gene in enumerate(best_chromosome) if gene == 1]),
    )


run(fitness1, population)
run(fitness2, population)


Fitness function: fitness1
Best solution: [1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0]
Fitness score: 1.508905852417303
Total energy: 304
Mean Reliability /100: 65.88888888888889
Mean Time to task completion /100: 43.666666666666664

Fitness function: fitness2
Best solution: [1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1]
Fitness score: 1.467741935483871
Total energy: 495
Mean Reliability /100: 45.5
Mean Time to task completion /100: 31.0
